# For N1 950 ; gl 1000 GeV ; 10k events ; using MLM

In [18]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file

home_dir = "root_files/MLM_N950/"  #/data/analysis/iduminic/Hammad/MSSM_LO_gogo_TEST_noCKKWMSSM_Neutralino_update/Events/run_10/
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Event")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(0).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(0).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f


Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 22021.42

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 22004.58

Number of events passed number of jets <= 4 cut: 17474.56

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 13152.23

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 11485.04

For SR MET > 200 GeV: S_Exp: 11485.04, S_Obs: 102274, and ratio: 0.11

For SR MET > 250 GeV: S_Exp: 9520.35, S_Obs: 41158, and ratio: 0.23

For SR MET > 300 GeV: S_Exp: 7263.76, S_Obs: 20893, and ratio: 0.35

For SR MET > 350 GeV: S_Exp: 5759.36, S_Obs: 11937, and ratio: 0.48

For SR MET > 400 GeV: S_Exp: 4457.05, S_Obs: 7214, and ratio: 0.62

For SR MET > 500 GeV: S_Exp: 2700.05, S_Obs: 2918, and ratio: 0.93

For SR MET > 600 GeV: S_Exp: 1582.98, S_Obs: 1391, and ratio: 1.14

For SR MET > 700 GeV: S_Exp: 959.89, S_Obs: 574, and ratio: 1.67

For SR MET > 800 GeV: S_Exp: 606.25, S_Obs: 298, and ratio: 2.03

Fo

# For N1 900 GeV ; gl 1000 GeV ; 10k events ; using CKKW

In [19]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-900-1000/Events/run_10k/tag_1_delphes_events.root

home_dir = "root_files/N900/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Event")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(0).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(0).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)


Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13018.45

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13007.07

Number of events passed number of jets <= 4 cut: 11713.59

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9723.18

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8024.32

For SR MET > 200 GeV: S_Exp: 8024.32, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 4768.36, S_Obs: 41158, and ratio: 0.12

For SR MET > 300 GeV: S_Exp: 3542.42, S_Obs: 20893, and ratio: 0.17

For SR MET > 350 GeV: S_Exp: 2907.19, S_Obs: 11937, and ratio: 0.24

For SR MET > 400 GeV: S_Exp: 2393.91, S_Obs: 7214, and ratio: 0.33

For SR MET > 500 GeV: S_Exp: 1520.09, S_Obs: 2918, and ratio: 0.52

For SR MET > 600 GeV: S_Exp: 784.91, S_Obs: 1391, and ratio: 0.56

For SR MET > 700 GeV: S_Exp: 160.48, S_Obs: 574, and ratio: 0.28

For SR MET > 800 GeV: S_Exp: 93.42, S_Obs: 298, and ratio: 0.31

For SR 

# For N1 950 GeV ; gl 1000 GeV ; 10k events

In [20]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-950-1000/Events/run_10k/tag_1_delphes_events.root

home_dir = "root_files/N950/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Event")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(0).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(0).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)


Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 12742.56

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 12728.26

Number of events passed number of jets <= 4 cut: 11589.52

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9262.20

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7692.95

For SR MET > 200 GeV: S_Exp: 7692.95, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 4350.17, S_Obs: 41158, and ratio: 0.11

For SR MET > 300 GeV: S_Exp: 3118.67, S_Obs: 20893, and ratio: 0.15

For SR MET > 350 GeV: S_Exp: 2471.10, S_Obs: 11937, and ratio: 0.21

For SR MET > 400 GeV: S_Exp: 1887.26, S_Obs: 7214, and ratio: 0.26

For SR MET > 500 GeV: S_Exp: 1230.40, S_Obs: 2918, and ratio: 0.42

For SR MET > 600 GeV: S_Exp: 768.09, S_Obs: 1391, and ratio: 0.55

For SR MET > 700 GeV: S_Exp: 183.26, S_Obs: 574, and ratio: 0.32

For SR MET > 800 GeV: S_Exp: 81.63, S_Obs: 298, and ratio: 0.27

For SR 

# For N1 990 GeV ; gl 1000 GeV ; 10k events

In [21]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-990-1000/Events/run_10k/tag_1_delphes_events.root

home_dir = "root_files/N990/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Event")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(0).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(0).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)


Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 12628.94

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 12616.05

Number of events passed number of jets <= 4 cut: 11454.59

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9472.88

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7913.62

For SR MET > 200 GeV: S_Exp: 7913.62, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 4542.52, S_Obs: 41158, and ratio: 0.11

For SR MET > 300 GeV: S_Exp: 3570.31, S_Obs: 20893, and ratio: 0.17

For SR MET > 350 GeV: S_Exp: 2864.87, S_Obs: 11937, and ratio: 0.24

For SR MET > 400 GeV: S_Exp: 2447.59, S_Obs: 7214, and ratio: 0.34

For SR MET > 500 GeV: S_Exp: 1580.73, S_Obs: 2918, and ratio: 0.54

For SR MET > 600 GeV: S_Exp: 793.00, S_Obs: 1391, and ratio: 0.57

For SR MET > 700 GeV: S_Exp: 143.72, S_Obs: 574, and ratio: 0.25

For SR MET > 800 GeV: S_Exp: 55.22, S_Obs: 298, and ratio: 0.19

For SR 

# For N1 850 GeV ; gl 1000 GeV ; 10k events ; using matching

In [23]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-850-1000/Events/run_10k/tag_1_delphes_events.root

home_dir = "root_files/N850/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Event")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(0).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(0).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            # Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                delta_phis.append(delta_phi_jet_met)

            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(0).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)


Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 12320.58

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 12306.28

Number of events passed number of jets <= 4 cut: 11171.73

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9362.07

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7694.01

For SR MET > 200 GeV: S_Exp: 7694.01, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 4614.73, S_Obs: 41158, and ratio: 0.11

For SR MET > 300 GeV: S_Exp: 3453.25, S_Obs: 20893, and ratio: 0.17

For SR MET > 350 GeV: S_Exp: 2821.29, S_Obs: 11937, and ratio: 0.24

For SR MET > 400 GeV: S_Exp: 2287.18, S_Obs: 7214, and ratio: 0.32

For SR MET > 500 GeV: S_Exp: 1530.02, S_Obs: 2918, and ratio: 0.52

For SR MET > 600 GeV: S_Exp: 800.84, S_Obs: 1391, and ratio: 0.58

For SR MET > 700 GeV: S_Exp: 181.81, S_Obs: 574, and ratio: 0.32

For SR MET > 800 GeV: S_Exp: 88.60, S_Obs: 298, and ratio: 0.30

For SR 